In [ ]:
import boto3
from pprint import pprint

### Prerequisito: Crear versión de Guardrail

In [ ]:
# SDK
bedrock_client = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')

In [ ]:
# Our Guardrail ID - CHANGE ME
guardrail_id = "qx2mggr6cvk5"

In [ ]:
# Create Version
response = bedrock_client.create_guardrail_version(
    guardrailIdentifier=guardrail_id,
    description='Bloqueamos preguntas sobre Salarios y temas distintos a RRHH'
)

In [ ]:
pprint(response)

In [ ]:
# Get Guardrail version
guardrail_version = response.get("version")

#### Probamos Guardrail

In [ ]:
# Elige tu modelo, acorde a tu región: https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html 
model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"

In [ ]:
def ask_hr_chatbot(system_prompt, user_prompt, model_id, guardrail_id, guardrail_version):
  """_summary_

  Args:
      system_prompt (_type_): _description_
      user_prompt (_type_): _description_
      model_id (_type_): _description_
      guardrail_id (_type_): _description_
      guardrail_version (_type_): _description_

  Returns:
      _type_: _description_
  """
  response = bedrock_runtime.converse(
                modelId=model_id,
                system=[{
                  "text": system_prompt
                }],
                messages=[{
                  "role": "user",
                  "content": [{"text": user_prompt}]
                }],
                inferenceConfig={
                    "maxTokens": 4096,
                    "temperature": 0.5
                },
                guardrailConfig={
                  'guardrailIdentifier': guardrail_id,
                  'guardrailVersion': guardrail_version,
                  'trace': 'enabled'
              }
            )
  
  return response

In [ ]:
# System Prompt
system_prompt = "You are a Human Resource assistant from our company."

# Question
user_prompt = "How much my colleague Justin earns per month?"

In [ ]:
response = ask_hr_chatbot(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    model_id=model_id,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

In [ ]:
pprint(response['output']['message']['content'][0]['text'])
print("\n"+"#"*150+"\n")
pprint(response['trace'])

In [ ]:
# Question
user_prompt = "How can I improve my health habits, to improve my daily routines?"

In [ ]:
response = ask_hr_chatbot(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    model_id=model_id,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

In [ ]:
pprint(response['output']['message']['content'][0]['text'])
print("\n"+"#"*150+"\n")
pprint(response['trace'])

#### Contextual Grounding

In [ ]:
def ask_hr_chatbot_with_grounding(system_prompt, user_prompt, model_id, kb_retrieved_chunks, 
                                  guardrail_id, guardrail_version):
  """_summary_

  Args:
      system_prompt (_type_): _description_
      user_prompt (_type_): _description_
      model_id (_type_): _description_
      kb_retrieved_chunks (_type_): _description_
      guardrail_id (_type_): _description_
      guardrail_version (_type_): _description_

  Returns:
      _type_: _description_
  """
  response = bedrock_runtime.converse(
    modelId=model_id,
    system=[{
      "text": system_prompt
    }],
    messages=[
    {
        "role": "user",
        "content": [
            {
                "guardContent": {
                    "text": {
                        "text": kb_retrieved_chunks,
                        "qualifiers": ["grounding_source"],
                    }
                }
            },
            {
                "guardContent": {
                    "text": {
                        "text": user_prompt,
                        "qualifiers": ["query"],
                    }
                }
            },
        ],
        }
    ],
    inferenceConfig={
    "maxTokens": 4096,
    "temperature": 0.5
    },
    guardrailConfig={
        "guardrailIdentifier": guardrail_id,
        "guardrailVersion": guardrail_version,
        "trace": "enabled"
        }
    )

  return response

In [ ]:
# Question
user_prompt = "Who is Leonel Messi from our company?"

# System Prompt
system_prompt = """
You are a virtual assistant in Data Quarks Labs. 
Respond company-related questions, including information from our wiki, perks or holidays.
"""

# Knowledge Base Retrieval
kb_retrieved_chunks = """Leonel Messi Contreras is the president of our company, Data Quarks Labs, 
which is a company that offers AI & Data services. 
We offer vacations of 35 days for all our employees."""

In [ ]:
response = ask_hr_chatbot_with_grounding(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    model_id=model_id,
    kb_retrieved_chunks=kb_retrieved_chunks,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

In [ ]:
pprint(response['output']['message']['content'][0]['text'])
print("\n"+"#"*150+"\n")
pprint(response['trace'])

In [ ]:
# Question
data_chunks = """
The document AUC-ROC, together with AUC-RWC, is created to highlight the policies related 
to the company car given to director roles.
"""

user_prompt = f"""Information relevant to answer question:
<data_chunks>
{data_chunks}
</data_chunks>

User question:
<question>
What is the AUC-ROC policy for?
</question>
"""

# System Prompt
system_prompt = """
You are an HR virtual assistant in Pharmaceutical X Labs."""

# Knowledge Base Retrieval
kb_retrieved_chunks = """Tech IT Message:
The VPN must be used at all times when connecting from home.
The laptop must be blocked at all times when not in use.
"""

In [ ]:
response = ask_hr_chatbot_with_grounding(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    model_id=model_id,
    kb_retrieved_chunks=kb_retrieved_chunks,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

In [ ]:
pprint(response['output']['message']['content'][0]['text'])
print("\n"+"#"*150+"\n")
pprint(response['trace'])

---
# No olvides borrar los Guardrails!

In [ ]:
response = bedrock_client.delete_guardrail(
    guardrailIdentifier=guardrail_id
)
print(response['ResponseMetadata']['HTTPStatusCode'])